In [8]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [12]:
basin = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_7")
gcp = ee.FeatureCollection("users/ujavalgandhi/e2e/arkavathy_gcps")
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

print('gcp size',gcp.size().getInfo())

# Accuracy Assessment
# Split the GCPs into 2 parts randomly
# Training Fraction - 60%
# Testing Fraction - 40%
gcp = gcp.randomColumn()
training_gcp = gcp.filter(ee.Filter.lt('random', 0.6))
testing_gcp = gcp.filter(ee.Filter.gte('random', 0.6))
print('training size',training_gcp.size().getInfo())
print('testing size',testing_gcp.size().getInfo())



arkavathy = basin.filter(ee.Filter.eq('HYBAS_ID', 4071139640))
geometry = arkavathy.geometry()

m = geemap.Map()

m.centerObject(geometry)

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Filter the image collection.
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.bounds(geometry))\
    .select('B.*')

composite = filtered.median()

# Display the input composite.
m.addLayer(composite.clip(geometry), rgbVis, 'image')

# Overlay the point on the image to get training data.
training = composite.sampleRegions(
  collection=training_gcp,
  properties=['landcover'],
  scale=10,
  tileScale=8
  )

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=composite.bandNames()
)

# Classify the image.
classified = composite.classify(classifier)

palette = ['#cc6d8f', '#ffc107', '#1e88e5', '#004d40' ]
m.addLayer(classified.clip(geometry), {'min': 0, 'max': 3, 'palette': palette}, '2019')

test = classified.sampleRegions(
  collection=testing_gcp,
  properties=['landcover'],
  scale=10,
  tileScale=8
  )

confusion_metrix = test.errorMatrix('landcover', 'classification')
print('Confusion Matrix', confusion_metrix.getInfo())

gcp size 447
training size 271
testing size 176
Confusion Matrix [[43, 3, 0, 0], [5, 36, 0, 0], [0, 0, 46, 2], [0, 0, 2, 39]]


In [15]:
# Calculate Producer's Accuracy
producer_accuracy = confusion_metrix.producersAccuracy()

# Calculate Consumer's Accuracy
consumer_accuracy = confusion_metrix.consumersAccuracy()

# Calculate F1 Score
f1_score = confusion_metrix.fscore()

# Print the assessment metrics
print("Producer's Accuracy:", producer_accuracy.getInfo())
print("Consumer's Accuracy:", consumer_accuracy.getInfo())
print("F1 Score:", f1_score.getInfo())

Producer's Accuracy: [[0.9347826086956522], [0.8780487804878049], [0.9583333333333334], [0.9512195121951219]]
Consumer's Accuracy: [[0.8958333333333334, 0.9230769230769231, 0.9583333333333334, 0.9512195121951219]]
F1 Score: [0.9148936170212766, 0.9, 0.9583333333333334, 0.9512195121951219]
